In [70]:
import json
with open('./test_decompose.json', 'r') as f:
    data = f.readlines()
print(len(data))
print(data[0])
output = []
pre_ins = "Based on the table, if the query is complex to answer, provide semantically complete sub-queries for knowledge that you need. Split the queries with ’;’."
for line in data:
    line = json.loads(line)
    output.append({'instruction': pre_ins + line['query'],
                   'input': '',
                   'output': line['pred'],
                   'history': []})
 
print(len(output))
with open('tabular_decompose.json' ,'w', encoding='utf-8') as f:
 
    json.dump(output, f, ensure_ascii=False, indent=4)
        # f.write(json.dumps(l) + '\n')   

100
{"query": "on the december 15 euro '84 qualifying game , the team participate against wale", "pred": "When did the team participate in the Euro '84 qualifying game?; Who did the team play against in the Euro '84 qualifying game on December 15?"}

100


### decompose

In [8]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from data_loader import TableFormat
query_examples = [
    # "after 2005 , the winner of the lifetime achievement award be andrew rule john silvester , sandra harvey lindsay simpson , marele day , shane maloney , and peter doyle",
                  "all 12 club play a total of 22 game for the wru division one east",
                  "a gamecube game loss the award in each of the first 3 year",
                  "polona hercog partner with alberta brianti after she have stephanie vogt as the partner",
                  ]
task_examples = ["query rewrite", "query decompose", "query ambiguity resolve"]
new_query_examples = [
    # "Who were the winners of the lifetime achievement award after 2005?;",
                      "How many clubs play for the wru division one east in total?; How many clubs play 22 game for the wru division one east?;",
                      "a gamecube game loss the award in each of the first 3 year",
                      "When did polona hercog partner with alberta brianti?; When did polona hercog partner with stephanie vogt?",
                      ]
num_k = 3
inds = [1, 6, 5]
table_loader = TableLoader(table_name='tabfact', split='validation', use_sample=True, small_test=False)
examples = [TableFormat(format='none', data=table_loader.dataset[inds[i]], use_sampling=True).format_html(table_loader.dataset[inds[i]]['table']['caption']) for i in range(num_k)]

examples_prompt = PromptTemplate(input_variables=["query", "table", "new_query"], template=
"""Query: {query}
Sub-Table: {table}
new_query: {new_query}""")

examples_dict = [{"query": query_examples[i],
                                    "table": examples[i],
                                    "new_query": new_query_examples[i]} for i in range(num_k)]
decompose_prompt = FewShotPromptTemplate(
    examples=examples_dict,
    example_prompt=examples_prompt,
    prefix="""You are capable of converting complex query into sub-queries. Based on the table, if the query is complex to answer, provide continuity sub-queries for knowledge that you need. 
Split the queries with ’;’.""",
    suffix=
    """Query: {query}
Sub-Table: {table}
new_query: 
    """,
    input_variables=["query", "table"],
)

# Sub-questions are separated by semicolons.
# answer_instruction = PromptTemplate(input_variables=["SQL", "table", "claim"], 
#                                     template="""
# Below is a sub-table generated by excuting the SQL. You need to understand the logic behind the SQL filtering and complete task using the final sub-table. 
# SQL Excuted: 
# ```{SQL}```
# Sub-table: {table}
# Query: {claim}
# answer the question given in the query. Only return the string instead of other format information. Do not repeat the question.
# """ )


In [2]:
from prompt_manager import get_k_shot_with_aug, get_k_shot_with_answer, view_instruction, row_instruction
import pandas as pd
from utils import parse_specific_composition
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI, OpenAI
from data_loader import TableFormat, TableLoader
from langchain.memory import ChatMessageHistory
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from sqlalchemy import create_engine
from executor import SQLManager
import sqlparse

In [4]:
task_name = 'tabfact'
split = 'test'
model_name = 'gpt-3.5-turbo-0125'
model = ChatOpenAI(model_name=model_name, openai_api_base="https://api.chatanywhere.tech/v1",
                       openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW", temperature=0.1)
schema_information = pd.read_csv(f"result/aug/{task_name}_{split}_schema.csv", index_col='table_id')
aug_information = pd.read_csv(f"result/aug/{task_name}_{split}_summary.csv", index_col='table_id')
composition_information = pd.read_csv(f"result/aug/{task_name}_{split}_composition.csv", index_col='table_id')
engine = create_engine('sqlite:///db/sqlite/tabfact.db', echo=False)
manager = SQLManager(engine=engine)

In [13]:
table_loader = TableLoader(table_name='tabfact', split='test', use_sample=False, small_test=True)
sample = table_loader.normalize_table(table_loader.dataset[2])

In [15]:
sample = table_loader.normalize_table(table_loader.dataset[2])
llm_chain = LLMChain(llm=model, prompt=stage_0_prompt, verbose=True)
stage_0_batch_pred = llm_chain.batch([{"query": sample['query']}], return_only_outputs=True)[0]['text'].split(':')[-1]
print(stage_0_batch_pred)
sub_queries = stage_0_batch_pred.split(';')



> Entering new LLMChain chain...
Prompt after formatting:
You are an expert at converting user questions into database queries. 
Your task is to effectively decompose complex, multihop questions into simpler, manageable/abstract sub-questions or tasks. This process involves breaking down a question that requires information from multiple sources or steps into broader, more abstract questions that can be answered individually. 


Query: Is the following query true or false? after 2005 , the winner of the lifetime achievement award be andrew rule john silvester , sandra harvey lindsay simpson , marele day , shane maloney , and peter doyle
new_query: Who were the winners of the lifetime achievement award after 2005?; Are the winners andrew rule john silvester , sandra harvey lindsay simpson , marele day , shane maloney , and peter doyle?


Query: what is the number of listings from barrington, farmington, and rochester combined?
new_query: What is the number of listings from barrington?

In [62]:
#维护一个Agent Memory
Agent_history = []
agent_prompt = ChatPromptTemplate.from_messages(
    [
        # (
        #     "system",
        #     """"""
            
        #     # return 'A' if you think you need more information, else return 'B'. You should only use information during the conversation.,
        # ),
        MessagesPlaceholder(variable_name="chat_history"),
    ]
)
system =  PromptTemplate(input_variables=["history"], template="""Based on the history information, your task is to only based on the conversation information to answer the user query. 
Note: Do not use information on your own, only use information from the conversation history!

conversation histroy:
{history}

The output should choose from Choice A and Choice B:
Choice A: ###If you cannot get the answer from conversation histroy, reorganize the question and return the question explicitly.
Choice B: ###If you are confident in the answer, answer it directly.""")
chain = LLMChain(llm=model, prompt=system, verbose=True)
Agent_history.append('Q: Who were the winners of the lifetime achievement award after 2005?')
# Agent_history.append('A: the winners are andrew rule john silvester , sandra harvey lindsay simpson , marele day , shane maloney , and peter doyle')
# Agent_history.append('Q: Are the winners andrew rule john silvester , sandra harvey lindsay simpson , marele day , shane maloney , and peter doyle?')


In [63]:
chain.batch(inputs=[{"history": '\n'.join(Agent_history)}], return_only_outputs=True)



> Entering new LLMChain chain...
Prompt after formatting:
Based on the history information, your task is to only based on the conversation information to answer the user query. 
Note: Do not use information on your own, only use information from the conversation history!

conversation histroy:
Q: Who were the winners of the lifetime achievement award after 2005?

The output should choose from Choice A and Choice B:
Choice A: ###If you cannot get the answer from conversation histroy, reorganize the question and return the question explicitly.
Choice B: ###If you are confident in the answer, answer it directly.

> Finished chain.


[{'text': 'Choice A: The conversation history does not provide information on the winners of the lifetime achievement award after 2005.'}]

### step-back

In [92]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from data_loader import TableFormat
query_examples = ["the country of ecuador be the only country that use the us dollar (usd) as its currency , and the central bank be the federal reserve",]
task_examples = ["query rewrite", "query decompose", "query ambiguity resolve"]
new_query_examples = ["Which country uses the US dollar as its currency and has the Federal Reserve as its central bank?",
                    #   "what is the number of listings from barrington?; what is the number of listings from farmington?; what is the number of listings from rochester combined?",
                      ]
examples_prompt = PromptTemplate(input_variables=["query", "new_query"], template=
"""
Query: {query}
new_query: {new_query}""")
num_k = 1
examples_dict = [{"query": query_examples[i],
                                    "new_query": new_query_examples[i]} for i in range(num_k)]
step_back_prompt = FewShotPromptTemplate(
    examples=examples_dict,
    example_prompt=examples_prompt,
    prefix="""Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer.""",
    suffix=
    """
Query: {query}
    """,
    input_variables=["query"],
)


In [93]:
def get_k_shot_with_answer(k: int=1):
    sqls = ["SELECT MIN(points) FROM DF WHERE rider = 'roger dutton / tony wright';"]
    thoughts = ["Based on the SQL query provided, the minimum number of points that Roger Dutton / Tony Wright received in the 1972 Isle of Man TT event was 3. Therefore, the claim that 2 is the fewest points they received is false. The output should be 0."]
    tables = ["<table>\n<caption>1972 isle of man tt</caption>\n<thead>\n<tr><th>  MIN(points)</th></tr>\n</thead>\n<tbody>\n<tr><td>3            </td></tr>\n</tbody>\n</table>"]
    claims = ["2 be the fewest point that roger dutton / tony wright receive"]
    # inds from test split
    examples_prompt = PromptTemplate(input_variables=["SQL", "table", "claim", "thought", "output"], template=
    """
    SQL Excuted: 
    ```{SQL}```
    Sub-table: {table}
    ...

    Query: {claim}
    Thought: {thought}
    Output: {output}
    """)
    examples_dict = dict(zip(["SQL", "table", "claim", "thought", "output"], [sqls[0], tables[0], claims[0], thoughts[0], '0']))
    prompt_template = FewShotPromptTemplate(
        examples=[examples_dict],
        example_prompt=examples_prompt,
        prefix="""Below are some sub-tables generated by excuting the SQL. You need to understand the logic behind the SQL filtering. Complete task using all information below. Please think step by step and only return 0 or 1 without any other information at last.""",
        suffix=
        """
    {information}
    Query: {query}
    Thought: """,
        input_variables=["table", "query"],
)
    return prompt_template

In [94]:
def scene_A(query, sample):
    formatter = TableFormat(format='none', data=sample, use_sampling=True)
    k_shot_prompt = get_k_shot_with_aug()
    with get_openai_callback() as cb:
        llm_chain = LLMChain(llm=model, prompt=k_shot_prompt, verbose=False)
        summary_aug, column_aug = aug_information.loc[sample['id']]['summary'], aug_information.loc[sample['id']]['column_description'] 
        stage_1_batch_pred = llm_chain.batch([dict({'table': formatter.format_html(table_caption=sample['table']['caption']),
                                            'claim': query,
                                            'aug':  summary_aug
                                            })], return_only_outputs=True)[0]['text']
        print(stage_1_batch_pred)
        # stage 2: SQL generation
        llm_chain = LLMChain(llm=model, prompt=row_instruction, verbose=False)
        columns = [formatter.normalize_col_name(c.strip()) for c in stage_1_batch_pred.split(',')]

        #     # formatter.normalize_schema(schema_information.loc[sample['id']]['schema'])
        formatter.data = formatter.data.loc[:, columns]
        
        extra_information = '\n'.join(parse_specific_composition(composition_information.loc[sample['id']]['composition'], formatter.data.columns))
        stage_2_batch_pred = llm_chain.batch([dict({'table': formatter.format_html(table_caption=sample['table']['caption']),
                                            'claim': query,
                                            'aug':  extra_information
                                            })], return_only_outputs=True)[0]['text']
    print("total_tokens:", cb.total_tokens)
    print(stage_2_batch_pred)
    # stage 3: SQL Excution
    formatter.normalize_schema(schema_information.loc[sample['id']]['schema'])
    formatter.data = manager.execute_from_df(stage_2_batch_pred, formatter.all_data, table_name='DF')
    return query, stage_2_batch_pred, formatter.format_html()

In [101]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI, OpenAI
model = ChatOpenAI(model_name='gpt-3.5-turbo-0125', openai_api_base="https://api.chatanywhere.tech/v1",
                       openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW", temperature=0.1)
table_loader = TableLoader(table_name='tabfact', split='test', use_sample=False, small_test=True)
outputs = []
labels = []
muilti_answer_instruction = PromptTemplate(input_variables=["information", "claim"], 
                                    template="""
Below are some sub-tables generated by excuting the SQL. You need to understand the logic behind the SQL filtering. Complete task using all information below. 
{information}
Query: {query}
verify whether the provided claim/query is true or false, return 0 if it's false, or 1 if it's true. Please think step by step and return 0/1 at last.
""" )
for i in range(10, 20):
    sample = table_loader.normalize_table(
                        table_loader.dataset[i])
    labels.append(sample['label'])
    all_queries = [sample['query']]
    formatter = TableFormat(format='none', data=sample, use_sampling=True)
    llm_chain = LLMChain(llm=model, prompt=step_back_prompt, verbose=False)
    batch_pred = llm_chain.batch([{"query": sample['query']}], return_only_outputs=True)
    all_queries.append(batch_pred[0]['text'].split(':')[-1])
    print(batch_pred[0]['text'].split(':')[-1])

    args_list = [{"query": q, "sample": sample} for q in all_queries]
    results = parallel_run_kwargs(scene_A, args_list)
    temp = [f"""
    SQL Excuted: 
    ```{res[1]}```
    Sub-table: {res[2]}""" for res in results]
    llm_chain = LLMChain(llm=model, prompt=muilti_answer_instruction, verbose=True)
    batch_pred = llm_chain.batch([{"query": sample['query'], "information": '\n'.join(temp)}], return_only_outputs=True)
    print(batch_pred[0]['text'])
    outputs.append(batch_pred[0]['text'])
# llm_chain = LLMChain(llm=model, prompt=decompose_prompt, verbose=False)
# batch_pred = llm_chain.batch([{"query": sample['query'], "table": formatter.format_html(table_caption=sample['table']['caption'])}], return_only_outputs=True)
# print(batch_pred[0]['text'].split(':')[-1].split(';'))
# all_queries.extend(batch_pred[0]['text'].split(':')[-1].split(';'))
# "Is the following query true or false?" +

 How many goals did four players score in total, all of whom are in league 4?
player, league, total
player, total, league
total_tokens: 1255
SELECT SUM(total) AS total_goals
FROM DF
WHERE league = 4;
total_tokens: 1253
SELECT player
FROM DF
WHERE total = 3 AND league = 4;


> Entering new LLMChain chain...
Prompt after formatting:

Below are some sub-tables generated by excuting the SQL. You need to understand the logic behind the SQL filtering. Complete task using all information below. 

    SQL Excuted: 
    ```SELECT player
FROM DF
WHERE total = 3 AND league = 4;```
    Sub-table: <table>
<thead>
<tr></tr>
</thead>
<tbody>
</tbody>
</table>

    SQL Excuted: 
    ```SELECT SUM(total) AS total_goals
FROM DF
WHERE league = 4;```
    Sub-table: <table>
<thead>
<tr><th>  total_goals</th></tr>
</thead>
<tbody>
<tr><td>12           </td></tr>
</tbody>
</table>
Query: 4 player score 3 goal total , and all 4 be in league 4
verify whether the provided claim/query is true or false, return 0 

In [99]:
outputs

['Based on the SQL queries provided, there are a total of 12 goals scored in league 4. However, there is no information about 4 players scoring 3 goals each in league 4. Therefore, the claim that 4 players scored 3 goals each in league 4 is false. The output should be 0. \n\nOutput: 0',
 'Based on the SQL queries provided, there are no players who meet the criteria of playing in the Belgian First Division A league, having a title playoff greater than 0, and being the next highest ranked player after Matías Suárez. Therefore, the claim that Jonathan Legear scored 4 more goals than Matías Suárez is false. The output should be 0. \n\nOutput: 0',
 'Based on the SQL queries provided, we can see that Jonathan Legear is not included in the results of the first query, and the player with the maximum total goals is not Jonathan Legear. Additionally, the second query only includes Mbark Boussoufa from the list of players mentioned. Therefore, the claim that Jonathan Legear scored more goals than

In [105]:
print(outputs[0])

To verify the claim, we need to follow these steps:

1. From the first SQL query, we know that there are players who have scored a total of 3 goals and are in league 4.
2. From the second SQL query, we know that the total number of goals scored by all players in league 4 is 12.

Now, let's check if there are 4 players who have scored 3 goals each in league 4:

If there are 4 players who have scored 3 goals each in league 4, the total number of goals scored by these 4 players would be 4 * 3 = 12.

Since the total number of goals scored by all players in league 4 is 12, and the claim is that 4 players scored 3 goals each in league 4, the claim is TRUE.

Therefore, the final answer is 1.


In [86]:
import concurrent.futures
from langchain_community.callbacks import get_openai_callback
def parallel_run(func, args_list):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = [executor.submit(func, arg) for arg in args_list]
        return [future.result() for future in concurrent.futures.as_completed(results)]

def parallel_run_kwargs(func, args_list):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(lambda kwargs: func(**kwargs), args_list)
        return list(results)
# args_list = [{"query": 'Who took the loss in the game on August 30 and who suffered the loss in the game on August 31?', "sample": sample},{"query": sample['query'], "sample": sample}]
args_list = [{"query": q, "sample": sample} for q in all_queries]

results = parallel_run_kwargs(scene_A, args_list)
print(results)
# print(cb.total_tokens)


winning_team, score
year, winning_team, score
total_tokens: 1321
SELECT MIN(CAST(SUBSTR(score, 1, INSTR(score, ' - ') - 1) AS REAL)) AS min_score, 
       MAX(CAST(SUBSTR(score, INSTR(score, ' - ') + 3) AS REAL)) AS max_score
FROM DF;
total_tokens: 1353
SELECT * FROM DF WHERE winning_team = 'europe' AND score >= '13' AND score <= '18';
[('the highest score for a winning team be 18 , while the lowest score for a winning team be 13', "SELECT * FROM DF WHERE winning_team = 'europe' AND score >= '13' AND score <= '18';", '<table>\n<thead>\n<tr><th>  year</th><th>                                 venue</th><th>  winning_team</th><th>    score</th><th>  usa_captain</th><th>  europe_captain</th></tr>\n</thead>\n<tbody>\n<tr><td>2011  </td><td>killeen castle golf resort , ireland  </td><td>europe        </td><td>15 - 13  </td><td>rosie jones  </td><td>alison nicholas </td></tr>\n<tr><td>2003  </td><td>barsebäck golf & country club , sweden</td><td>europe        </td><td>17½ - 10½</td><td>patty 

In [90]:
muilti_answer_instruction = PromptTemplate(input_variables=["information", "claim"], 
                                    template="""
Below are some sub-tables generated by excuting the SQL. You need to understand the logic behind the SQL filtering. Complete task using all information below. 
{information}
Query: {query}
verify whether the provided claim/query is true or false, return 0 if it's false, or 1 if it's true. Please think step by step.
""" )
temp = [f"""
SQL Excuted: 
```{res[1]}```
Sub-table: {res[2]}""" for res in results]
muilti_answer_instruction = get_k_shot_with_answer()
llm_chain = LLMChain(llm=model, prompt=muilti_answer_instruction, verbose=True)
batch_pred = llm_chain.batch([{"query": sample['query'], "information": '\n'.join(temp)}], return_only_outputs=True)



> Entering new LLMChain chain...
Prompt after formatting:
Below are some sub-tables generated by excuting the SQL. You need to understand the logic behind the SQL filtering. Complete task using all information below. Please think step by step and only return 0 or 1 without any other information at last.


    SQL Excuted: 
    ```SELECT MIN(points) FROM DF WHERE rider = 'roger dutton / tony wright';```
    Sub-table: <table>
<caption>1972 isle of man tt</caption>
<thead>
<tr><th>  MIN(points)</th></tr>
</thead>
<tbody>
<tr><td>3            </td></tr>
</tbody>
</table>
    ...

    Query: 2 be the fewest point that roger dutton / tony wright receive
    Thought: Based on the SQL query provided, the minimum number of points that Roger Dutton / Tony Wright received in the 1972 Isle of Man TT event was 3. Therefore, the claim that 2 is the fewest points they received is false. The output should be 0.
    Output: 0
    


    
SQL Excuted: 
```SELECT * FROM DF WHERE winning_team = 'europe

In [81]:
results[1]

(' When did Galina Voskoboeva play the same opponent more than once?',
 'SELECT opponent, COUNT(opponent) \nFROM DF\nWHERE opponent IN (\n    SELECT opponent\n    FROM DF\n    GROUP BY opponent\n    HAVING COUNT(opponent) > 1\n)\nGROUP BY opponent;',
 '<table>\n<thead>\n<tr></tr>\n</thead>\n<tbody>\n</tbody>\n</table>')

In [91]:
print(batch_pred[0]['text'])

Based on the SQL query provided, the highest score for a winning team is 13, while the lowest score for a winning team is 11. Therefore, the claim that the highest score for a winning team is 18 and the lowest score is 13 is false. The output should be 0. 

Output: 0


In [ ]:

1. From the first sub-table, we see that on August 30, the opponent was "athletics" and the loss was "mohler (1 - 10)".
2. From the second sub-table, we see that "mohler (1 - 10)" is listed as a loss for the opponent "athletics".
3. Therefore, the claim that on August 30, mohler (1 - 10) took the loss is TRUE.

4. Next, from the first sub-table, we see that on August 31, the opponent was "athletics" and the loss was "oquist (2 - 5)".
5. From the second sub-table, we see that "oquist (2 - 5)" is listed as a loss for the opponent "athletics".
6. Therefore, the claim that on August 31, oquist (2 - 5) suffered the loss is TRUE.

In [35]:
def scene_B():
    agent_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Based on the history information, your task is to only based on the conversation information to answer the user query.
    If you cannot get the answer from past history, reorganize the question and return the question explicitly. If you are confident in the answer, answer it directly.""",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
    ]
)
    chain = LLMChain(llm=model, prompt=agent_prompt, verbose=True)
    return chain.invoke(
    {
        "chat_history": Agent_history.messages,
    }
)['text']

In [39]:
result = [ ]
for i in range(len(table_loader.dataset)):
    sample = table_loader.normalize_table(table_loader.dataset[i])
    llm_chain = LLMChain(llm=model, prompt=stage_0_prompt, verbose=False)
    stage_0_batch_pred = llm_chain.batch([{"query": sample['query']}], return_only_outputs=True)[0]['text'].split(':')[-1]
    print(stage_0_batch_pred)
    sub_queries = stage_0_batch_pred.split(';')

    from langchain_community.callbacks import get_openai_callback
    Agent_history = ChatMessageHistory()
    agent_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """Based on the history information, your task is to determine whether the information is enough to answer the user query.
                return 'A' if you think you need more information, else return 'B'. You should only use information during the conversation.""",
            ),
            MessagesPlaceholder(variable_name="chat_history"),
        ]
    )
    chain = LLMChain(llm=model, prompt=agent_prompt, verbose=True)
    with get_openai_callback() as cb:
        for sub_query in sub_queries:
            Agent_history.add_user_message(sub_query)
            choice = chain.invoke(
            {
                "chat_history": Agent_history.messages,
            }
            )['text']
            if 'A' in choice:
                Agent_history.add_ai_message(scene_A(sub_query, sample))
            else:
                res = scene_B()
                print(res)
                result.append(res)
                
    print(cb.total_tokens)
    

 What is the mintage of the 31997 release?; When was the 31997 release released?


> Entering new LLMChain chain...
Prompt after formatting:
System: Based on the history information, your task is to determine whether the information is enough to answer the user query.
                return 'A' if you think you need more information, else return 'B'. You should only use information during the conversation.
Human:  What is the mintage of the 31997 release?

> Finished chain.
year, theme, mintage
SELECT mintage
FROM DF
WHERE theme = 'hmcs bras dor' AND mintage = '31997';
Yes, the 31997 mintage was released in 2001.


> Entering new LLMChain chain...
Prompt after formatting:
System: Based on the history information, your task is to determine whether the information is enough to answer the user query.
                return 'A' if you think you need more information, else return 'B'. You should only use information during the conversation.
Human:  What is the mintage of the 31997 release?


OperationalError: (sqlite3.OperationalError) no such column: season
[SQL: SELECT record
FROM DF
WHERE season = 2005
AND team = 'Milwaukee Brewers';]
(Background on this error at: https://sqlalche.me/e/20/e3q8)